In [21]:
import cv2
import numpy as np
from skimage.morphology import closing, disk
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.draw import polygon
from skimage.feature import canny
from scipy.ndimage.morphology import distance_transform_edt
from skimage.transform import resize
import pandas as pd
from utils.process_images import *
from utils.process_data import *

C:\Users\Admin\AppData\Local\Temp\ipykernel_7108\3012124106.py:8: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import distance_transform_edt


In [22]:
def process_single_image(image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    thresh_img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    clean_img = closing(thresh_img > 0, disk(1))

    # Calculate stroke thickness
    thickness = stroke_thickness(clean_img)
    mean_thickness = np.mean(thickness)
    std_thickness = np.std(thickness)

    # Calculate number of pixels, end points, and intersection points
    num_pixels = np.sum(clean_img)
    num_ep = number_of_end_points(clean_img, k_nn)
    num_inters = number_of_intersection_points(clean_img, k_nn)

    # Calculate interactions between features
    interactions = {
        'mean_thickness_std_thickness': mean_thickness * std_thickness,
        'mean_thickness_num_pixels': mean_thickness * num_pixels,
        'mean_thickness_num_ep': mean_thickness * num_ep,
        'mean_thickness_num_inters': mean_thickness * num_inters,
        'std_thickness_num_pixels': std_thickness * num_pixels,
        'std_thickness_num_ep': std_thickness * num_ep,
        'std_thickness_num_inters': std_thickness * num_inters,
        'num_pixels_num_ep': num_pixels * num_ep,
        'num_pixels_num_inters': num_pixels * num_inters,
        'num_ep_num_inters': num_ep * num_inters
    }

    # Create a DataFrame with the processed features
    feature_data = {
        'mean_thickness': mean_thickness,
        'std_thickness': std_thickness,
        'num_pixels': num_pixels,
        'num_ep': num_ep,
        'num_inters': num_inters,
        **interactions  # Include interactions in the DataFrame
    }

    df = pd.DataFrame([feature_data])

    return df



In [30]:
image_path = 'images\V03PO01.png'  # Replace with the path to your single image
processed_df = process_single_image(image_path)
processed_df

ValueError: setting an array element with a sequence.

In [24]:
import pickle

# Load the RF model
with open('rf_model.pickle', 'rb') as model_file:
    loaded_rf_model = pickle.load(model_file)



In [25]:
# Make predictions using the loaded model
predictions = loaded_rf_model.predict(processed_df )

# The 'predictions' variable now contains the predicted labels for each row in processed_df
print(predictions)


[1]


In [28]:
from PIL import Image
def get_image_size(image_path):
    try:
        img = Image.open(image_path)
        width, height = img.size
        return width, height
    except IOError as e:
        print(f"Unable to open image: {e}")
        return None, None

 # Replace with the actual path to your image
width, height = get_image_size('drawings\\wave\\training\\healthy\\V01HO02.png')

if width and height:
    print(f"Image width: {width}px")
    print(f"Image height: {height}px")

Image width: 512px
Image height: 265px


In [31]:
from PIL import Image
def get_image_size(image_path):
    try:
        img = Image.open(image_path)
        width, height = img.size
        return width, height
    except IOError as e:
        print(f"Unable to open image: {e}")
        return None, None

 # Replace with the actual path to your image
width, height = get_image_size('images\V03PO01.png')

if width and height:
    print(f"Image width: {width}px")
    print(f"Image height: {height}px")

Image width: 512px
Image height: 263px
